In [2]:
# chapter 5 & 6

## 5.1请求方式

### 5.1.1GET请求

In [3]:
import requests    # 导入网络请求模块requests
import json
from lxml import etree  # 导入lxml模块
# 导入requests.exceptions模块中的三种异常类
from requests.exceptions import ReadTimeout,HTTPError,RequestException
import pandas as pd  # 导入pandas模块

In [4]:
import requests_cache
import time
from requests_html import HTMLSession,UserAgent    # 导入HTMLSession类
from requests_html import AsyncHTMLSession
import urllib3
import re

In [5]:
# 发送网络请求 不带参数的网络请求
#HTTP中的GET，POST，PUT，DELETE就对应着对这个资源的 查，改，增，删 4个操作

response = requests.get('https://www.baidu.com')
print('响应状态码为：',response.status_code)  # 打印状态码
print('请求的网络地址为：',response.url)          # 打印请求url
print('头部信息为：',response.headers)      # 打印头部信息
print('cookie信息为：',response.cookies)      # 打印cookie信息

KeyboardInterrupt: 

### 5.1.2对应结果进行utf-8编码

In [ ]:
response = requests.get('https://www.baidu.com/')
response.encoding='utf-8'    # 对响应结果进行utf-8编码
print(response.text)         # 以文本形式打印网页源码

### 5.1.3 爬取二进制数据

In [ ]:
response = requests.get('https://www.baidu.com/img/bd_logo1.png?where=super')
print(response.content)                # 打印二进制数据
with open('百度logo.png','wb')as f:   # 通过open函数将二进制数据写入本地文件
    f.write(response.content)          # 写入

### 5.1.5GET带参数请求

In [ ]:
response=requests.get('http://httpbin.org/get?name=Jack&age=30')
print(response.text)

In [ ]:
data={'name':'Michael','age':'36'}
response=requests.get('http://httpbin.org/get',params=data)
print(response.text)

### 5.1.5 POST请求

In [ ]:
#POST请求时需要设置请求参数data
# 字典类型的表单参数
data = {'1': '能力是有限的，而努力是无限的。',
        '2':'星光不问赶路人，时光不负有心人。'}
# 发送网络请求
response = requests.post('http://httpbin.org/post',data=data)
response_dict = json.loads(response.text)      # 将响应数据转换为字典类型
print(response_dict)

In [ ]:
#POST 中的参数格式可以是列表 元组 JSON
#元组
data1 = (('1', '能力是有限的，而努力是无限的。'),
        ('2','星光不问赶路人，时光不负有心人。'))
#列表
data2 = [('1', '能力是有限的，而努力是无限的。'),
        ('2','星光不问赶路人，时光不负有心人。')]
#字典        
data3 = {'1': '能力是有限的，而努力是无限的。',
        '2':'星光不问赶路人，时光不负有心人。'}
# 发送网络请求
response1 = requests.post('http://httpbin.org/post',data=data1)
response_1 = json.loads(response1.text)      # 将响应数据转换为字典类型
print(response_1)
response2 = requests.post('http://httpbin.org/post',data=data2)
response_2 = json.loads(response2.text)      # 将响应数据转换为字典类型
print(response_2)
response3 = requests.post('http://httpbin.org/post',data=data3)
response_3 = json.loads(response3.text)      # 将响应数据转换为字典类型
print(response_3)

## 5.2复杂的网络请求

### 5.2.1添加header

In [ ]:
#header模拟浏览器
url = 'https://www.baidu.com/'     # 创建需要爬取网页的地址
# 创建头部信息
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0'}
response  = requests.get(url, headers=headers)    # 发送网络请求
print(response.status_code)    

### 5.2.2验证cookies

In [ ]:
cookies = 'bid=q4OuPU0Qkac; gr_user_id=0de5911b-87c8-4382-bccc-a4f1f4024ce1; ll="108089"; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1634288780%2C%22https%3A%2F%2Fwww.google.com%2F%22%5D; _pk_ses.100001.8cb4=*; __utma=30149280.1731075480.1633327591.1633327591.1634288795.2; __utmc=30149280; __utmz=30149280.1634288795.2.2.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmt=1; dbcl2="247781609:iC9Qe6IPneo"; ck=mlay; ap_v=0,6.0; push_noty_num=0; push_doumail_num=0; __utmv=30149280.24778; _pk_id.100001.8cb4=4ec46b7583de2448.1634288780.1.1634288875.1634288780.; __utmb=30149280.5.10.1634288795'
headers = {'Host': 'www.douban.com',
           'Referer': 'https://www.hao123.com/',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                         'AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/72.0.3626.121 Safari/537.36'}
# 创建RequestsCookieJar对象，用于设置cookies信息
cookies_jar = requests.cookies.RequestsCookieJar()
for cookie in cookies.split(';'):
    key, value = cookie.split('=', 1)
    cookies_jar.set(key, value)  # 将cookies保存RequestsCookieJar当中
# 发送网络请求
response = requests.get('https://www.douban.com/',
headers=headers, cookies=cookies_jar)
if response.status_code == 200:  # 请求成功时
    html = etree.HTML(response.text)  # 解析html代码
    # 获取用户名
    name = html.xpath('//*[@id="db-global-nav"]/div/div[1]/ul/li[2]/a/span[1]/text()')
    print(name[0])  # 打印用户名

### 5.2.3 会话请求

In [ ]:
#session第一次登入 第二次发送请求就不需要设置cookies
s = requests.Session()  # 创建会话对象
data={'username': 'mrsoft', 'password': 'mrsoft'}  # 创建用户名、密码的表单数据
# 发送登录请求
response =s.post('http://site2.rjkflm.com:666/index/index/chklogin.html',data=data)
response2=s.get('http://site2.rjkflm.com:666')   # 发送登录后页面请求
print('登录信息：',response.text)                # 打印登录信息
print('登录后页面信息如下:\n',response2.text)    # 打印登录后的页面信息


### 5.2.4 验证请求

In [ ]:
#auth 验证参数
from requests.auth import HTTPBasicAuth  # 导入HTTPBasicAuth类
# 定义请求地址
url = 'http://sck.rjkflm.com:666/spider/auth/'
ah = HTTPBasicAuth('admin','admin')      # 创建HTTPBasicAuth对象，参数为用户名与密码
response = requests.get(url=url,auth=ah) # 发送网络请求
if response.status_code==200:            # 如果请求成功
    print(response.text)                 # 打印验证后的HTML代码

### 5.2.5 网络超时与异常

In [ ]:
# 循环发送请求50次
for a in range(0, 10):
    try:    # 捕获异常
        # 设置超时为0.5秒
        response = requests.get('https://www.baidu.com/', timeout=0.01)
        print(response.status_code)                                        # 打印状态码
    except Exception as e:                                                 # 捕获异常
        print('异常'+str(e))    

In [ ]:
#识别网络异常分类 3中常见的网络异常
# 循环发送请求50次
for a in range(0, 50):
    try:    # 捕获异常
        # 设置超时为0.5秒
        response = requests.get('https://www.baidu.com/', timeout=0.01)
        print(response.status_code)                                        # 打印状态码
    except ReadTimeout:                                                     # 超时异常
        print('timeout')
    except HTTPError:                                                       # HTTP异常
        print('httperror')
    except RequestException:                                               # 请求异常
        print('reqerror')

### 5.2.6上传文件

In [ ]:
#requests post()参数  files
bd = open('百度logo.png','rb')     # 读取指定文件
file = {'file':bd}                 # 定义需要上传的图片文件
# 发送上传文件的网络请求
response = requests.post('http://httpbin.org/post',files = file)
print(response.text)               # 打印响应结果

## 5.3 代理服务

### 5.3.1 代理应用

In [ ]:
# 头部信息
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                         'AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/72.0.3626.121 Safari/537.36'}
proxy = {'http': 'http://117.88.176.38:3000',
         'https': 'https://117.88.176.38:3000'}  # 设置代理ip与对应的端口号
try:
    # 对需要爬取的网页发送请求,verify=False不验证服务器的SSL证书
    response = requests.get('http://2020.ip138.com', headers= headers,proxies=proxy,verify=False,timeout=3)
    print(response.status_code)  # 打印响应状态码
except Exception as e:
    print('错误异常信息为：',e)    # 打印异常信息

### 5.3.2 获取免费的代理IP

In [ ]:
# 头部信息
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                         'AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/72.0.3626.121 Safari/537.36'}
# 发送网络请求
response = requests.get('https://www.xicidaili.com/nn/', headers=headers)
response.encoding = 'utf-8'  # 设置编码方式
if response.status_code == 200:  # 判断请求是否成功
    html = etree.HTML(response.text)  # 解析HTML
    table = html.xpath('//table[@id="ip_list"]')[0]  # 获取table标签内容
    trs = table.xpath('//tr')[1:]  # 获取所有tr标签，排除第一条
    ip_table = pd.DataFrame(columns=['ip'])      # 创建临时表格数据
    ip_list = []       # 创建保存ip地址的列表
    # 循环遍历标签内容
    for t in trs:
        ip = t.xpath('td/text()')[0]  # 获取代理ip
        port = t.xpath('td/text()')[1]  # 获取端口
        ip_list.append(ip+':'+port)     # 将ip与端口组合并添加至列表当中
        print('代理ip为：', ip, '对应端口为：', port)
    ip_table['ip']=ip_list              # 将提取的ip保存至excel文件中的ip列
    # 生成xlsx文件
    ip_table.to_excel('ip.xlsx', sheet_name='data')

In [ ]:
ip_table = pd.read_excel('ip.xlsx')  # 读取代理IP文件内容
ip = ip_table['ip']                      # 获取代理ip列信息
# 头部信息
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                         'AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/72.0.3626.121 Safari/537.36'}
# 循环遍历代理IP并通过代理发送网络请求
for i in ip:
    proxies = {'http': 'http://{ip}'.format(ip=i),
               'https': 'https://{ip}'.format(ip=i)}
    try:
        # verify=False不验证服务器的SSL证书
        response = requests.get('http://2020.ip138.com/',
                                headers=headers,proxies=proxies,verify=False,timeout=2)
        if response.status_code==200:   # 判断请求是否成功,请求成功说明代理IP可用
            response.encoding='utf-8'     # 进行编码
            html = etree.HTML(response.text)  # 解析HTML
            info = html.xpath('/html/body/p[1]/text()')
            print(info[0].strip())                 # 输出当前ip匿名信息
    except Exception as e:
        pass
        print('错误异常信息为：',e)    # 打印异常信息

In [ ]:
import requests_cache
import time
from requests_html import HTMLSession,UserAgent    # 导入HTMLSession类
from requests_html import AsyncHTMLSession
import urllib3
import re

## 6.1requests-Cache的安装与测试

In [ ]:
version = requests_cache.__version__
version

### 6.2缓存的应用

In [ ]:
#requests_cache install_cashe()
requests_cache.install_cache()    # 设置缓存
requests_cache.clear()            # 清理缓存
url = 'http://httpbin.org/get'    # 定义测试地址
r = requests.get(url)             # 第一次发送网络请求
print('是否存在缓存：',r.from_cache)  # False表示不存在缓存
r = requests.get(url)             # 第二次发送网络请求
print('是否存在缓存：',r.from_cache)  # True表示存在缓存


In [ ]:
#设置延时 
#hook判断是否存在缓存
requests_cache.install_cache()    # 设置缓存
requests_cache.clear()            # 清理缓存
# 定义钩子函数
def make_throttle_hook(timeout=0.1):
    def hook(response, *args, **kwargs):
        print(response.text)       #  打印请求结果
        # 判断没有缓存时就添加延时
        if not getattr(response, 'from_cache', False):
            print('等待',timeout,'秒！')
            time.sleep(timeout)     # 等待指定时间
        else:
            print('是否存在请求缓存！',response.from_cache)  # 存在缓存输出True
        return response
    return hook

if __name__ == '__main__':
    requests_cache.install_cache()              # 创建缓存
    requests_cache.clear()                      # 清理缓存
    s = requests_cache.CachedSession()          # 创建缓存会话
    s.hooks = {'response': make_throttle_hook(5)}  # 配置钩子函数
    s.get('http://httpbin.org/get')            # 模拟发送第一次网络请求
    s.get('http://httpbin.org/get')            # 模拟发送第二次网络请求


### 6.6 Requests-HTML模块

In [ ]:
session = HTMLSession()          # 创建HTML会话对象
url = 'http://news.youth.cn/'    # 定义请求地址
r =session.get(url)              # 发送网络请求
print(r.html)                    # 打印网络请求的url地址

In [ ]:
session = HTMLSession()          # 创建HTML会话对象
data = {'user':'admin','password':123456}    # 模拟表单登录的数据
r = session.post('http://httpbin.org/post',data=data)   # 发送post请求
if r.status_code == 200:                                # 判断请求是否成功
    print(r.text)                                        # 以文本形式打印返回结果

In [ ]:
session = HTMLSession()          # 创建HTML会话对象
ua = UserAgent(verify_ssl=False).random          # 创建随机请求头
r = session.get('http://httpbin.org/get',headers = {'user-agent': ua})
if r.status_code == 200:         # 判断请求是否成功
    print(r.text)                # 以文本形式打印返回结果

### 6.3.2 数据的获取

In [ ]:
session = HTMLSession()          # 创建HTML会话对象
ua = UserAgent().random          # 创建随机请求头
# 发送网路请求
r = session.get('http://news.youth.cn/jsxw/index.htm',
                headers = {'user-agent': ua})
r.encoding='gb2312'              # 编码
if r.status_code == 200:         # 判断请求是否成功
    # 获取所有class=tj3_1中的li标签
    li_all = r.html.xpath('.//ul[@class="tj3_1"]/li')
    for li in li_all:                 # 循环遍历每个li标签
        news_title = li.find('a')[0].text  # 提取新闻标题内容
        # 获取新闻详情对应的地址
        news_href = 'http://news.youth.cn/jsxw'+\
                    li.find('a[href]')[0].attrs.get('href').lstrip('.')
        news_time = li.find('font')[0].text    # 获取新闻发布的时间
        print('新闻标题为：',news_title)   # 打印新闻标题
        print('新闻url地址为：',news_href) # 打印新闻url地址
        print('新闻发布时间为：',news_time)# 打印新闻发布时间

In [ ]:
#find()方法中的containing参数
session = HTMLSession()          # 创建HTML会话对象
ua = UserAgent().random          # 创建随机请求头
# 发送网路请求
r = session.get('http://news.youth.cn/jsxw/index.htm',
                headers = {'user-agent': ua})
r.encoding='gb2312'              # 编码
if r.status_code == 200:         # 判断请求是否成功
    # 获取所有class=tj3_1中的li标签
    li_all = r.html.xpath('.//ul[@class="tj3_1"]/li')
    for li in r.html.find('li',containing='美国'):                 # 循环遍历每个li标签
        news_title = li.find('a')[0].text  # 提取新闻标题内容
        # 获取新闻详情对应的地址
        news_href = 'http://news.youth.cn/jsxw'+\
                    li.find('a[href]')[0].attrs.get('href').lstrip('.')
        news_time = li.find('font')[0].text    # 获取新闻发布的时间
        print('新闻标题为：',news_title)   # 打印新闻标题
        print('新闻url地址为：',news_href) # 打印新闻url地址
        print('新闻发布时间为：',news_time)# 打印新闻发布时间

In [ ]:
#search（）方法与search_all方法
session = HTMLSession()          # 创建HTML会话对象
ua = UserAgent().random          # 创建随机请求头
# 发送网路请求
r = session.get('http://news.youth.cn/jsxw/index.htm',
                headers = {'user-agent': ua})
r.encoding='gb2312'              # 编码
if r.status_code == 200:         # 判断请求是否成功
    # 获取所有class=tj3_1中的li标签
    li_all = r.html.xpath('.//ul[@class="tj3_1"]/li')
    for li in r.html.find('li',containing='美国'):                 # 循环遍历每个li标签
        a= li.search('<a href="{}">{}</a>')
        news_title=a[1]
        news_href='http://news.youth.cn/jsxw'+a[0]
        news_time=li.search('<font>{}</font>')[0]
        print('新闻标题为：',news_title)   # 打印新闻标题
        print('新闻url地址为：',news_href) # 打印新闻url地址
        print('新闻发布时间为：',news_time)# 打印新闻发布时间

In [ ]:
# search_all
session = HTMLSession()          # 创建HTML会话对象
ua = UserAgent().random          # 创建随机请求头
# 发送网路请求
r = session.get('http://news.youth.cn/jsxw/index.htm',
                headers = {'user-agent': ua})
r.encoding='gb2312'              # 编码
if r.status_code == 200:         # 判断请求是否成功
    # 获取所有class=tj3_1
    class_tj3_1 = r.html.xpath('.//ul[@class="tj3_1"]/li')
    #使用search_all方法获取所有获取所有class=tj3_1中的li标签
    li_all=class_tj3_1[0].search_all('<li>{}<li>')
    for li in r.html.find('li',containing='美国'):                 # 循环遍历每个li标签
        #通过re
        a= re.findall('<font>(.*?)</font><a href="(.*?)">(.*?)</a>',li[0])
        news_title=a[0][2]
        news_href='http://news.youth.cn/jsxw'+a[0][1]
        news_time=li.search('<font>{}</font>')[0][0]
        print('新闻标题为：',news_title)   # 打印新闻标题
        print('新闻url地址为：',news_href) # 打印新闻url地址
        print('新闻发布时间为：',news_time)# 打印新闻发布时间

### 6.3.3 获取动态加载的数据

In [ ]:
from requests_html import AsyncHTMLSession
#session = HTMLSession()          # 创建HTML会话对象
ua = UserAgent().random          # 创建随机请求头
asession = AsyncHTMLSession()
# 发送网路请求
r = await session.get('https://movie.douban.com/tag/#/?sort=U&range=0,10'
                '&tags=%E7%94%B5%E5%BD%B1,2020',headers = {'user-agent': ua})
r.encoding='gb2312'              # 编码
if r.status_code == 200:         # 判断请求是否成功
    await r.html.render()              # 调用render()方法，没有Chromium浏览器就自动下载
    class_wp = r.html.xpath('.//div[@class="list-wp"]/a')  # 获取当前页面中所有电影信息的a标签
    for a in class_wp:
        title = a.find('p span')[0].text  # 获取电影名称
        rate = a.find('p span')[1].text  # 获取电影评分
        details_url = a.attrs.get('href')  # 获取详情页url地址
        img_url = a.find('img')[0].attrs.get('src')  # 获取图片url地址
        print('电影名称为：', title)  # 打印电影名称
        print('电影评分为：', rate)  # 打印电影评分
        print('详情页地址为：', details_url)  # 打印电影详情页url地址
        print('图片地址为：', img_url)  # 打印电影图片地址

In [ ]:
from requests_html import HTMLSession,UserAgent    # 导入HTMLSession类

session = HTMLSession()          # 创建HTML会话对象
ua = UserAgent().random          # 创建随机请求头
# 发送网路请求
r = session.get('https://movie.douban.com/tag/#/?sort=U&range=0,10'\
                '&tags=%E7%94%B5%E5%BD%B1,2020',headers = {'user-agent': ua})
r.encoding='gb2312'              # 编码
if r.status_code == 200:         # 判断请求是否成功
    r.html.render()              # 调用render()方法，没有Chromium浏览器就自动下载
    class_wp = r.html.xpath('.//div[@class="list-wp"]/a')  # 获取当前页面中所有电影信息的a标签
    for a in class_wp:
        title = a.find('p span')[0].text  # 获取电影名称
        rate = a.find('p span')[1].text  # 获取电影评分
        details_url = a.attrs.get('href')  # 获取详情页url地址
        img_url = a.find('img')[0].attrs.get('src')  # 获取图片url地址
        print('电影名称为：', title)  # 打印电影名称
        print('电影评分为：', rate)  # 打印电影评分
        print('详情页地址为：', details_url)  # 打印电影详情页url地址
        print('图片地址为：', img_url)  # 打印电影图片地址

In [ ]:
from requests_html import HTMLSession,UserAgent    # 导入HTMLSession类
asession = AsyncHTMLSession()
#session = HTMLSession()          # 创建HTML会话对象
ua = UserAgent().random          # 创建随机请求头
# 发送网路请求
r = asession.get('https://movie.douban.com/tag/#/?sort=U&range=0,10'
                '&tags=%E7%94%B5%E5%BD%B1,2020',headers = {'user-agent': ua})
r.encoding='gb2312'              # 编码
if r.status_code == 200:         # 判断请求是否成功
    r.html.render()              # 调用render()方法，没有Chromium浏览器就自动下载
    class_wp = r.html.xpath('.//div[@class="list-wp"]/a')  # 获取当前页面中所有电影信息的a标签
    for a in class_wp:
        title = a.find('p span')[0].text  # 获取电影名称
        rate = a.find('p span')[1].text  # 获取电影评分
        details_url = a.attrs.get('href')  # 获取详情页url地址
        img_url = a.find('img')[0].attrs.get('src')  # 获取图片url地址
        print('电影名称为：', title)  # 打印电影名称
        print('电影评分为：', rate)  # 打印电影评分
        print('详情页地址为：', details_url)  # 打印电影详情页url地址
        print('图片地址为：', img_url)  # 打印电影图片地址

In [6]:
from requests_html import HTMLSession,UserAgent    # 导入HTMLSession类
from bs4 import BeautifulSoup
from requests_html import AsyncHTMLSession
ua = UserAgent().random          # 创建随机请求头
#session = HTMLSession()          # 创建HTML会话对象
asession = AsyncHTMLSession()

# 发送网路请求
r = await asession.get('https://movie.douban.com/tag/#/?sort=U&range=0,10'
                '&tags=%E7%94%B5%E5%BD%B1,2020',headers = {'user-agent': ua})
r.encoding='gb2312'              # 编码
if r.status_code == 200:         # 判断请求是否成功
    await r.html.arender()
    #r.html.render()              # 调用render()方法，没有Chromium浏览器就自动下载
    class_wp = r.html.xpath('.//div[@class="list-wp"]/a')  # 获取当前页面中所有电影信息的a标签
    for a in class_wp:
        title = a.find('p span')[0].text  # 获取电影名称
        rate = a.find('p span')[1].text  # 获取电影评分
        details_url = a.attrs.get('href')  # 获取详情页url地址
        img_url = a.find('img')[0].attrs.get('src')  # 获取图片url地址
        print('电影名称为：', title)  # 打印电影名称
        print('电影评分为：', rate)  # 打印电影评分
        print('详情页地址为：', details_url)  # 打印电影详情页url地址
        print('图片地址为：', img_url)  # 打印电影图片地址

电影名称为： 八佰
电影评分为： 7.5
详情页地址为： https://movie.douban.com/subject/26754233/
图片地址为： https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2615992304.jpg
电影名称为： 我和我的家乡
电影评分为： 7.1
详情页地址为： https://movie.douban.com/subject/35051512/
图片地址为： https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2620453443.jpg
电影名称为： 送你一朵小红花
电影评分为： 7.3
详情页地址为： https://movie.douban.com/subject/35096844/
图片地址为： https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2618247457.jpg
电影名称为： 姜子牙
电影评分为： 6.7
详情页地址为： https://movie.douban.com/subject/25907124/
图片地址为： https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2621219978.jpg
电影名称为： 夺冠
电影评分为： 7.2
详情页地址为： https://movie.douban.com/subject/30128916/
图片地址为： https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2620083313.jpg
电影名称为： 信条
电影评分为： 7.6
详情页地址为： https://movie.douban.com/subject/30444960/
图片地址为： https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2612061299.jpg
电影名称为： 心灵奇旅
电影评分为： 8.7
详情页地址为： https://movie.douban.com/subject/24